In [4]:
# Import the required libraries and dependencies
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path
import yfinance as yf

# Read the stock file to a dataframe
# Set the date column as the DataTimeIndex

cryptocurrencies = ['BTC-USD']
start_date = "2019-03-14"
end_date = "2022-07-01"
btc_df = yf.download(cryptocurrencies, start_date, end_date)



# Review the DataFrame
btc_df.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-03-14,3905.576904,3946.504395,3901.296875,3924.369141,3924.369141,10480789570
2019-03-15,3926.663330,3968.542969,3914.015381,3960.911133,3960.911133,9394210605
2019-03-16,3963.900146,4077.036377,3961.657471,4048.725830,4048.725830,9856166973
2019-03-17,4047.719482,4054.122070,4006.411133,4025.229004,4025.229004,8221625400
2019-03-18,4029.968506,4071.556641,4009.117188,4032.507324,4032.507324,9646954186


In [5]:
btc_df.describe()

,Open,High,Low,Close,Adj Close,Volume
count,1206.000000,1206.000000,1206.000000,1206.000000,1206.000000,1.206000e+03
mean,25286.670845,25932.328962,24559.999288,25295.756790,25295.756790,3.336179e+10
std,18558.422162,19038.727061,17989.256696,18543.559252,18543.559252,1.901841e+10
min,3905.576904,3946.504395,3901.296875,3924.369141,3924.369141,8.221625e+09
25%,9185.192139,9307.242676,9058.696777,9187.076660,9187.076660,2.100098e+10
50%,14483.743652,15543.824707,14262.645996,15062.328125,15062.328125,2.991718e+10
75%,41691.411133,42539.860352,40671.218750,41707.004883,41707.004883,4.012650e+10
max,67549.734375,68789.625000,66382.062500,67566.828125,67566.828125,3.509679e+11


In [6]:
btc_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1206 entries, 2019-03-14 to 2022-07-01
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       1206 non-null   float64
 1   High       1206 non-null   float64
 2   Low        1206 non-null   float64
 3   Close      1206 non-null   float64
 4   Adj Close  1206 non-null   float64
 5   Volume     1206 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 66.0 KB


In [7]:
# Slice to just the `close` column
signals_df = btc_df.loc[:,["Adj Close"]]

In [8]:
# Set the short window and long windows
short_window = 30
long_window = 100

In [9]:
# Generate the short and long moving averages (30 and 100 days, respectively)
signals_df['SMA30'] = signals_df['Adj Close'].rolling(window=short_window).mean()
signals_df['SMA100'] = signals_df['Adj Close'].rolling(window=long_window).mean()

# Prepopulate the `Signal` for trading
signals_df['Signal'] = 0.0

In [10]:
# Generate the trading signal 0 or 1,
# where 1 is when short-window (SMA30) is greater than the long (SMA100)
# and 0 otherwise
signals_df['Signal'][short_window:] = np.where(
    signals_df['SMA30'][short_window:] > signals_df['SMA100'][short_window:], 1.0, 0.0
)

In [11]:
# Calculate the points in time when the Signal value changes
# Identify trade entry (1) and exit (-1) points
signals_df['Entry/Exit'] = signals_df['Signal'].diff()

# Review the DataFrame
signals_df.tail(10)

,Adj Close,SMA30,SMA100,Signal,Entry/Exit
Date,,,,,
2022-06-22,19987.029297,26821.694661,35315.369336,0.0,0.0
2022-06-23,21085.876953,26536.037695,35132.840254,0.0,0.0
2022-06-24,21231.656250,26258.347526,34933.717520,0.0,0.0
2022-06-25,21502.337891,25999.517969,34739.227109,0.0,0.0
2022-06-26,21027.294922,25746.175326,34531.488496,0.0,0.0
2022-06-27,20735.478516,25476.861263,34316.936758,0.0,0.0
2022-06-28,20280.634766,25171.350521,34107.264863,0.0,0.0
2022-06-29,20104.023438,24783.938281,33897.525137,0.0,0.0
2022-06-30,19784.726562,24383.685482,33671.784316,0.0,0.0


In [12]:
# Visualize exit position relative to close price
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Adj Close'].hvplot.scatter(
    color='yellow',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize entry position relative to close price
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Adj Close'].hvplot.scatter(
    color='purple',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = signals_df[['Adj Close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = signals_df[['SMA30', 'SMA100', 'Adj Close']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Create the overlay plot
entry_exit_plot = security_close * moving_avgs * entry * exit

# Show the plot with a title
entry_exit_plot.opts(
    title="Bitcoin - SMA30, SMA100, Entry and Exit Points During The Pandemic"
)

:Overlay
   .Curve.I              :Curve   [Date]   (Adj Close)
   .NdOverlay.I          :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.Adj_Close.I  :Scatter   [Date]   (Adj Close)
   .Scatter.Adj_Close.II :Scatter   [Date]   (Adj Close)

In [13]:
# Set initial capital
initial_capital = float(50000)

# Set the share size
share_size = 0.02

In [14]:
# Buy a 500 share position when the dual moving average crossover Signal equals 1
# Otherwise, `Position` should be zero (sell)
signals_df['Position'] = share_size * signals_df['Signal']

In [15]:
# Determine the points in time where a share position is bought or sold
signals_df['Entry/Exit Position'] = signals_df['Position'].diff()

In [16]:
# Multiply the close price by the number of shares held, or the Position
signals_df['Portfolio Holdings'] = signals_df['Adj Close'] * signals_df['Position']

In [17]:
# Subtract the amount of either the cost or proceeds of the trade from the initial capital invested
signals_df['Portfolio Cash'] = initial_capital - (signals_df['Adj Close'] * signals_df['Entry/Exit Position']).cumsum() 

In [18]:
# Calculate the total portfolio value by adding the portfolio cash to the portfolio holdings (or investments)
signals_df['Portfolio Total'] = signals_df['Portfolio Cash'] + signals_df['Portfolio Holdings']

In [19]:
# Calculate the portfolio daily returns
signals_df['Portfolio Daily Returns'] = signals_df['Portfolio Total'].pct_change()

In [20]:
# Calculate the portfolio cumulative returns
signals_df['Portfolio Cumulative Returns'] = (1 + signals_df['Portfolio Daily Returns']).cumprod() - 1

In [21]:
# Print the DataFrame
signals_df.head(150)

,Adj Close,SMA30,SMA100,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns
Date,,,,,,,,,,,,
2019-03-14,3924.369141,NaN,NaN,0.0,NaN,0.00,NaN,0.000000,NaN,NaN,NaN,NaN
2019-03-15,3960.911133,NaN,NaN,0.0,0.0,0.00,0.0,0.000000,50000.000000,50000.000000,NaN,NaN
2019-03-16,4048.725830,NaN,NaN,0.0,0.0,0.00,0.0,0.000000,50000.000000,50000.000000,0.000000,0.000000
2019-03-17,4025.229004,NaN,NaN,0.0,0.0,0.00,0.0,0.000000,50000.000000,50000.000000,0.000000,0.000000
2019-03-18,4032.507324,NaN,NaN,0.0,0.0,0.00,0.0,0.000000,50000.000000,50000.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08-06,11478.168945,10618.120703,9164.186006,1.0,0.0,0.02,0.0,229.563379,49797.108867,50026.672246,-0.000131,0.000533
2019-08-07,11941.968750,10606.654395,9231.132168,1.0,0.0,0.02,0.0,238.839375,49797.108867,50035.948242,0.000185,0.000719
2019-08-08,11966.407227,10586.407552,9297.288975,1.0,0.0,0.02,0.0,239.328145,49797.108867,50036.437012,0.000010,0.000729


In [22]:
signals_df.describe()

,Adj Close,SMA30,SMA100,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns
count,1206.000000,1177.000000,1107.000000,1206.000000,1205.000000,1206.000000,1205.000000,1206.000000,1205.000000,1205.000000,1204.000000,1204.000000
mean,25295.756790,25600.534846,26035.578084,0.544776,0.000000,0.010896,0.000000,311.040530,49993.999670,50305.298325,0.000008,0.006111
std,18543.559252,18406.683302,17977.504015,0.498198,0.091135,0.009964,0.001823,410.890105,331.364314,347.479954,0.000384,0.006950
min,3924.369141,4429.301595,6333.660903,0.000000,-1.000000,0.000000,-0.020000,0.000000,49694.607354,49932.735117,-0.002968,-0.001345
25%,9187.076660,9253.727669,8971.061792,0.000000,0.000000,0.000000,0.000000,0.000000,49765.796025,50000.000000,-0.000003,0.000000
50%,15062.328125,14972.661361,15348.377061,1.000000,0.000000,0.020000,0.000000,185.348008,49797.108867,50071.614072,0.000000,0.001441
75%,41707.004883,41886.624219,42452.116943,1.000000,0.000000,0.020000,0.000000,456.092842,50000.676309,50626.860010,0.000035,0.012537
max,67566.828125,62792.418880,55005.481484,1.000000,1.000000,0.020000,0.020000,1351.336563,50636.546260,51045.943916,0.002886,0.020919


In [23]:
# Visualize exit position relative to total portfolio value
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Portfolio Total'].hvplot.scatter(
    color='yellow',
    marker='v',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize entry position relative to total portfolio value
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Portfolio Total'].hvplot.scatter(
    color='purple',
    marker='^',
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize the value of the total portfolio
total_portfolio_value = signals_df[['Portfolio Total']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    xlabel='Date',
    width=1000,
    height=400
)

# Overlay the plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(
    title="BTC Algorithm - Total Portfolio Value",
    yformatter='%.0f'
)

:Overlay
   .Curve.I                    :Curve   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [Date]   (Portfolio Total)

In [24]:
# cumulative moving average
# on unweighted closing price
btc_df2 = signals_df.copy()
btc_df2['CMA'] = btc_df['Close'].expanding().mean()
btc_df2['Close'] = btc_df['Close']

btc_df2

,Adj Close,SMA30,SMA100,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns,CMA,Close
Date,,,,,,,,,,,,,,
2019-03-14,3924.369141,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,3924.369141,3924.369141
2019-03-15,3960.911133,NaN,NaN,0.0,0.0,0.0,0.0,0.0,50000.000000,50000.000000,NaN,NaN,3942.640137,3960.911133
2019-03-16,4048.725830,NaN,NaN,0.0,0.0,0.0,0.0,0.0,50000.000000,50000.000000,0.0,0.000000,3978.002035,4048.725830
2019-03-17,4025.229004,NaN,NaN,0.0,0.0,0.0,0.0,0.0,50000.000000,50000.000000,0.0,0.000000,3989.808777,4025.229004
2019-03-18,4032.507324,NaN,NaN,0.0,0.0,0.0,0.0,0.0,50000.000000,50000.000000,0.0,0.000000,3998.348486,4032.507324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-27,20735.478516,25476.861263,34316.936758,0.0,0.0,0.0,0.0,0.0,50480.067588,50480.067588,0.0,0.009601,25313.846869,20735.478516
2022-06-28,20280.634766,25171.350521,34107.264863,0.0,0.0,0.0,0.0,0.0,50480.067588,50480.067588,0.0,0.009601,25309.662986,20280.634766
2022-06-29,20104.023438,24783.938281,33897.525137,0.0,0.0,0.0,0.0,0.0,50480.067588,50480.067588,0.0,0.009601,25305.339365,20104.023438


In [25]:
# Visualize exit position relative to close price
exit = btc_df2[btc_df2['Entry/Exit'] == -1.0]['Close'].hvplot.scatter(
    color='yellow',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize entry position relative to close price
entry = btc_df2[btc_df2['Entry/Exit'] == 1.0]['Close'].hvplot.scatter(
    color='purple',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = btc_df2[['Close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = btc_df2[['CMA','Close']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Create the overlay plot
entry_exit_plot = security_close * moving_avgs * entry * exit

# Show the plot with a title
entry_exit_plot.opts(
    title="Bitcoin - CMA, Entry and Exit Points During The Pandemic"
)

:Overlay
   .Curve.I          :Curve   [Date]   (Close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.Close.I  :Scatter   [Date]   (Close)
   .Scatter.Close.II :Scatter   [Date]   (Close)